In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import datetime as dt 
import yfinance as yf

from sklearn.metrics import mean_squared_error , mean_absolute_error , explained_variance_score , r2_score
from sklearn.metrics import confusion_matrix , classification_report , accuracy_score , roc_auc_score , roc_curve
from sklearn.metrics import mean_poisson_deviance , mean_gamma_deviance , accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , BatchNormalization

from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [5]:
df = yf.download('BTC-USD' ,start='2014-09-17' , end='2024-01-03')

[*********************100%***********************]  1 of 1 completed


In [6]:
df.to_csv('BTC-USD.csv')

In [7]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...
2023-12-29,42614.644531,43124.324219,41424.062500,42099.402344,42099.402344,26000021055
2023-12-30,42091.753906,42584.125000,41556.226562,42156.902344,42156.902344,16013925945
2023-12-31,42152.097656,42860.937500,41998.253906,42265.187500,42265.187500,16397498810


In [8]:
df.shape

(3395, 6)

In [9]:
print("total number of days in the dataset : ", df.shape[0])
print("total number of features in the dataset : ", df.shape[1])

total number of days in the dataset :  3395
total number of features in the dataset :  6


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3395 entries, 2014-09-17 to 2024-01-02
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3395 non-null   float64
 1   High       3395 non-null   float64
 2   Low        3395 non-null   float64
 3   Close      3395 non-null   float64
 4   Adj Close  3395 non-null   float64
 5   Volume     3395 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 185.7 KB


In [11]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3.395000e+03
mean,14588.960941,14930.505102,14222.420439,14600.736431,14600.736431,1.655625e+10
std,16197.096721,16586.513285,15762.882025,16200.689676,16200.689676,1.908303e+10
min,176.897003,211.731003,171.509995,178.102997,178.102997,5.914570e+06
25%,911.960510,923.695007,897.231018,918.540985,918.540985,1.571455e+08
50%,8243.402344,8408.516602,8054.120117,8245.915039,8245.915039,1.154115e+10
75%,23789.396484,24261.601562,23277.829102,23807.059570,23807.059570,2.687238e+10
max,67549.734375,68789.625000,66382.062500,67566.828125,67566.828125,3.509679e+11


In [13]:
print("total number of null values in the dataset : ", df.isnull().sum().sum())
print("total number of NA values in the dataset : ", df.isna().sum().sum())

total number of null values in the dataset :  0
total number of NA values in the dataset :  0


In [14]:
startdate = df.index.min()
enddate = df.index.max()
print("start date of dataset : ", startdate)
print("end date of dataset : ", enddate)

start date of dataset :  2014-09-17 00:00:00
end date of dataset :  2024-01-02 00:00:00


In [18]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['Open'], name="stock_open"))
fig.add_trace(go.Scatter(x=df.index, y=df['Close'], name="stock_close"))
fig.add_trace(go.Scatter(x=df.index, y=df['Low'], name="stock_low"))
fig.add_trace(go.Scatter(x=df.index, y=df['Volume'], name="stock_volume"))
fig.update_layout(title_text="Time Series data with Rangeslider", xaxis_rangeslider_visible=True)
fig.show()


In [23]:
df['month_year'] = df['month_year'].astype(str)
fig = px.bar(df, x='month_year', y='Open', color='month', barmode='group', height=400)  
fig.update_layout(title_text='Monthwise comparision between open and close price')
fig.show()

In [25]:
closedf = df[ 'Close']
closedf

Date
2014-09-17      457.334015
2014-09-18      424.440002
2014-09-19      394.795990
2014-09-20      408.903992
2014-09-21      398.821014
                  ...     
2023-12-29    42099.402344
2023-12-30    42156.902344
2023-12-31    42265.187500
2024-01-01    44167.332031
2024-01-02    44957.968750
Name: Close, Length: 3395, dtype: float64